In [1]:
import sys
path = "/Users/matth/Desktop"
sys.path.insert(0,path)
from SpotipyConfig import SPOTIFY_USER_ID, SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET 

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

auth_manager = SpotifyClientCredentials(SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [3]:
import numpy as np

In [4]:
import pandas as pd

In [ ]:
#SPOTIFY_USER_ID = "carlypiekos"

In [ ]:
# # Get all users playlists
# playlists = sp.user_playlists(SPOTIFY_USER_ID,limit=50,offset=0)


# plList = []
# while playlists:
#     for i, playlist in enumerate(playlists['items']):
#     #     print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
#         plList.append({"offset": (i + 1+playlists['offset']),"uri": playlist['uri'], "name": playlist['name']})
#     if playlists['next']:
#         playlists = sp.next(playlists)
#     else:
#         playlists = None

In [ ]:
# playlists = sp.user_playlists(SPOTIFY_USER_ID,limit=50,offset=0)

In [ ]:
# playlists['items']

In [ ]:
# show playlists
# plList

In [12]:
#########
pl_uri_dict = {"Dinner": "1p8HMfpfMz59SEy7FeWpg2",  
               "DanceParty": "36J8V7oFQoZtLI4gjIqEgR", 
               "LateDance": "6CvgtN8Q7zAZdIpWAlCjjR",
#                "MattLibrary": "42ndan1IXwwZOH3BB26BCe"
              }
pl_num_dict = {"Dinner": 1,  
               "DanceParty": 2, 
               "LateDance": 3,
#                "MattLibrary": "42ndan1IXwwZOH3BB26BCe"
              }

In [13]:
# Pick playlist and populate trList with the tracks.
trList = []
for playlistKey in pl_uri_dict:
    plTracksObj = sp.playlist_tracks(pl_uri_dict[playlistKey], fields=None, limit=100, offset=0, market=None, additional_types=('track', ))

    
    while plTracksObj:
        for i, playlist in enumerate(plTracksObj['items']):
        #     print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
            trList.append({"id": playlist['track']['id'],
                           "external_ids": playlist["track"]["external_ids"],
                           "name": playlist["track"]['name'], 
                           "added_at": playlist["added_at"],
                           "artists": ",".join([artist["name"] for artist in playlist["track"]["artists"]]),
                          "playlist": playlistKey,
                          "playlist_num": pl_num_dict[playlistKey]})
        if plTracksObj['next']:
            plTracksObj = sp.next(plTracksObj)
        else:
            plTracksObj = None

In [14]:
trList

[{'id': '4o0LyB69tylqDG6eTGhmig',
  'external_ids': {'isrc': 'USSM12004811'},
  'name': 'Fire for You',
  'added_at': '2022-03-25T16:53:46Z',
  'artists': 'Cannons',
  'playlist': 'Dinner',
  'playlist_num': 1},
 {'id': '4gsR34XSIE2fUY4odwZqym',
  'external_ids': {'isrc': 'QM4DW1526291'},
  'name': 'Seventeen',
  'added_at': '2022-03-25T16:54:15Z',
  'artists': 'Sjowgren',
  'playlist': 'Dinner',
  'playlist_num': 1},
 {'id': '6RKGBcDtgzXx21zUjpeu6F',
  'external_ids': {'isrc': 'GBAYE1400934'},
  'name': 'Cold Cold Man',
  'added_at': '2022-03-25T16:55:48Z',
  'artists': 'Saint Motel',
  'playlist': 'Dinner',
  'playlist_num': 1},
 {'id': '1pnwoWbZcS77Dx0jQVYK8X',
  'external_ids': {'isrc': 'TCABY1492268'},
  'name': 'Till Sunrise (feat. Mammals)',
  'added_at': '2022-03-25T17:05:16Z',
  'artists': 'Goldroom,Mammals',
  'playlist': 'Dinner',
  'playlist_num': 1},
 {'id': '3pdHWxpeFJqai3skQcRr41',
  'external_ids': {'isrc': 'USTB11700116'},
  'name': 'Somebody to Love',
  'added_at': '2

In [ ]:
# # Pick playlist and populate trList with the tracks.
# This_One = plList[14]["uri"]
# plTracksObj = sp.playlist_tracks(This_One, fields=None, limit=100, offset=0, market=None, additional_types=('track', ))

# trList = []
# while plTracksObj:
#     for i, playlist in enumerate(plTracksObj['items']):
#     #     print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
#         trList.append({"id": playlist['track']['id'],
#                        "external_ids": playlist["track"]["external_ids"],
#                        "name": playlist["track"]['name'], 
#                        "added_at": playlist["added_at"],
#                        "artists": ",".join([artist["name"] for artist in playlist["track"]["artists"]])})
#     if plTracksObj['next']:
#         plTracksObj = sp.next(plTracksObj)
#     else:
#         plTracksObj = None

In [ ]:
# plTracksObj = sp.playlist_tracks(This_One, fields=None, limit=100, offset=0, market=None, additional_types=('track', ))


In [ ]:
# plTracksObj['items'][0]["track"]["external_ids"]

In [15]:
# Add audio features to each track in trList

for i in range(1,int(np.ceil(len(trList)/100)+1)):
    temp_audiofeats = sp.audio_features([tr["id"] for tr in trList[100*(i-1):(100*i)]])
#     print("--------------")
#     print([100*(i-1), 100*i])
    for feat in temp_audiofeats:
        trackInd = next((j for j, track in enumerate(trList) if track["id"] == feat["id"]),None)
#         print(trackInd)
        trList[trackInd]["afeats"]=feat

In [16]:
for track in trList:
    if "afeats" not in track:
        track["afeats"] = sp.audio_features(track["id"])[0]

In [17]:
tempdf = pd.DataFrame(trList)
tempdf[tempdf["afeats"].isnull()]

,id,external_ids,name,added_at,artists,playlist,playlist_num,afeats


In [20]:
trList[47]

{'id': '76YDsWGWezclVy9Z6sDGhf',
 'external_ids': {'isrc': 'US25T9920900'},
 'name': "Da' Dip",
 'added_at': '2022-03-25T16:58:48Z',
 'artists': 'Freak Nasty',
 'playlist': 'DanceParty',
 'playlist_num': 2,
 'afeats': {'danceability': 0.953,
  'energy': 0.428,
  'key': 8,
  'loudness': -17.199,
  'mode': 1,
  'speechiness': 0.298,
  'acousticness': 0.0079,
  'instrumentalness': 0.0302,
  'liveness': 0.224,
  'valence': 0.723,
  'tempo': 131.737,
  'type': 'audio_features',
  'id': '76YDsWGWezclVy9Z6sDGhf',
  'uri': 'spotify:track:76YDsWGWezclVy9Z6sDGhf',
  'track_href': 'https://api.spotify.com/v1/tracks/76YDsWGWezclVy9Z6sDGhf',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/76YDsWGWezclVy9Z6sDGhf',
  'duration_ms': 237000,
  'time_signature': 4}}

In [21]:
playlist_id = [track["playlist"] for track in trList]
playlist_num = [track["playlist_num"] for track in trList]
song_id = [track["id"] for track in trList] 
ext_id = [track["external_ids"] for track in trList]
songtitle = [track["name"] for track in trList]
artists = [track["artists"] for track in trList]
added_at = [track["added_at"] for track in trList]
danceability =[track["afeats"]["danceability"] for track in trList]
energy =[track["afeats"]["energy"] for track in trList]
key =[track["afeats"]["key"] for track in trList]
loudness =[track["afeats"]["loudness"] for track in trList]
speechiness =[track["afeats"]["speechiness"] for track in trList]
instrumentalness = [track["afeats"]["instrumentalness"] for track in trList]
valence = [track["afeats"]["valence"] for track in trList]
tempo = [track["afeats"]["tempo"] for track in trList]
duration_ms = [track["afeats"]["duration_ms"] for track in trList]

playlistFrame = pd.DataFrame({"playlist": playlist_id,
                                "song_id": song_id,
                              "external_ids": ext_id,
                             "songtitle": songtitle,
                             "artists": artists,
                             "added_at": added_at,
                            "danceability": danceability,
                             "energy": energy,
                             "key": key,
                             "loudness": loudness,
                             "speechiness": speechiness,
                             "instrumentalness": instrumentalness,
                              "valence": valence,
                              "tempo": tempo,
                            "duration_ms": duration_ms,
                             "duration_min": [dur/1000/60 for dur in duration_ms],
                             "duration_hr": [dur/1000/60/60 for dur in duration_ms]})

In [22]:
# playlistFrame[playlistFrame["energy"]==min(playlistFrame["energy"])]

playlistFrame.to_excel("AllSongs.xlsx")

In [23]:
playlistFrame

,playlist,song_id,external_ids,songtitle,artists,added_at,danceability,energy,key,loudness,speechiness,instrumentalness,valence,tempo,duration_ms,duration_min,duration_hr
0,Dinner,4o0LyB69tylqDG6eTGhmig,{'isrc': 'USSM12004811'},Fire for You,Cannons,2022-03-25T16:53:46Z,0.737,0.678,6,-7.011,0.0962,0.157000,0.698,102.051,231826,3.863767,0.064396
1,Dinner,4gsR34XSIE2fUY4odwZqym,{'isrc': 'QM4DW1526291'},Seventeen,Sjowgren,2022-03-25T16:54:15Z,0.612,0.725,7,-5.602,0.0416,0.078400,0.333,146.008,225667,3.761117,0.062685
2,Dinner,6RKGBcDtgzXx21zUjpeu6F,{'isrc': 'GBAYE1400934'},Cold Cold Man,Saint Motel,2022-03-25T16:55:48Z,0.613,0.661,9,-4.984,0.0263,0.000000,0.509,110.978,230773,3.846217,0.064104
3,Dinner,1pnwoWbZcS77Dx0jQVYK8X,{'isrc': 'TCABY1492268'},Till Sunrise (feat. Mammals),"Goldroom,Mammals",2022-03-25T17:05:16Z,0.528,0.737,0,-5.705,0.0296,0.692000,0.175,84.029,246149,4.102483,0.068375
4,Dinner,3pdHWxpeFJqai3skQcRr41,{'isrc': 'USTB11700116'},Somebody to Love,Abhi The Nomad,2022-03-25T17:07:22Z,0.827,0.525,6,-5.823,0.0309,0.006740,0.883,119.985,250000,4.166667,0.069444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,LateDance,2fgKuNvD7vFYRGwvHeIPU6,{'isrc': 'QMCE32100308'},Long Distance,JORDY,2022-04-07T14:51:22Z,0.802,0.601,1,-7.291,0.0485,0.000000,0.921,110.027,148186,2.469767,0.041163
159,LateDance,1nPd60em5Qwp0NPykFz5Xj,{'isrc': 'USWB12100319'},Happy Endings (feat. iann dior and UPSAHL),"Mike Shinoda,iann dior,UPSAHL",2022-04-07T14:52:12Z,0.652,0.876,7,-3.323,0.0359,0.000000,0.695,126.007,212513,3.541883,0.059031
160,LateDance,33kwEgLredDQ01xIvbDRaK,{'isrc': 'AUNE32000133'},Fallin' Apart (feat. Denzel Curry & Pell),"Young Franco,Denzel Curry,Pell",2022-04-07T14:52:42Z,0.748,0.799,1,-7.675,0.3600,0.000000,0.800,111.016,124494,2.074900,0.034582
161,LateDance,4nBsvvjN134KFVcvFMLWwK,{'isrc': 'QZ9H32100011'},she calls me daddy,KiNG MALA,2022-04-07T14:53:07Z,0.796,0.426,9,-7.994,0.0324,0.004460,0.572,100.006,183570,3.059500,0.050992


In [ ]:
###################################### END

In [ ]:
# param = "duration_min"
# maxdf = playlistFrame[playlistFrame[param]==max(playlistFrame[param])]
# for ind in maxdf.index:
#      print(maxdf['artists'][ind]+ " - " + maxdf['songtitle'][ind] + " has max value for "+ param+": "+str(maxdf[param][ind]))

# mindf = playlistFrame[playlistFrame[param]==min(playlistFrame[param])]
# for ind in mindf.index:
#      print(mindf['artists'][ind]+ " - " + mindf['songtitle'][ind] + " has min value for "+ param+": "+str(mindf[param][ind]))

In [ ]:
# plotFrame = playlistFrame.iloc[: , [4,5,8,9,10]]
# boxplot = plotFrame.boxplot()

In [ ]:
# plotFrame = playlistFrame.iloc[: , [6,7]]
# boxplot = plotFrame.boxplot()

In [ ]:
# plotFrame = playlistFrame.iloc[: , [12]]
# boxplot = plotFrame.boxplot()

In [ ]:
# result = filter(lambda track: track["afeats"]["danceability"]== 0.958, trList)
# print(list(result))

In [ ]:
# danceyframe = pd.DataFrame({"Danceability": Danceability})
# boxplot = danceyframe.boxplot("Danceability")

In [ ]:
# sp.track("1QEYxXsoFBKoK4PIDX6Yn3")

In [ ]:
# playlistFrame

In [ ]:
# danceFrame = playlistFrame.sort_values(by="duration_min", ascending=True)
# danceFrame.head(10)

In [ ]:
# Lyrics Genius. Overall sense is accuracy is low. Insufficient error handling if song "not found", per example below.

In [ ]:
# from SpotipyConfig import GENIUS_CLIENT_TOKEN

In [ ]:
# from lyricsgenius import Genius
# genius = Genius(GENIUS_CLIENT_TOKEN)
# # Remove section headers (e.g. [Chorus]) from lyrics when searching
# genius.remove_section_headers = True

In [ ]:
# help(genius)

In [ ]:
# artist = genius.search_artist("Andy Shauf", max_songs=3, sort="title")
# print(artist.songs)

In [ ]:
# artist.id

In [ ]:
# genius.response_format = 'plain,html'

In [ ]:
# song = genius.search_song("Fallin' Apart (feat. Denzel Curry & Pell)", "Young Franco,Denzel Curry,Pell")
# song.lyrics

In [ ]:
# song.id

In [ ]:
# genius.song(song.id)

In [ ]:
# song = genius.search_song("Fallin' Apart", "Young Franco")
# song.lyrics

In [ ]:
# genius.lyrics(song.id)

In [ ]:
# song.id

In [ ]:
# Musixmatch api https://developer.musixmatch.com/documentation/api-methods
# full call syntax here https://www.youtube.com/watch?v=WFRzKmpepj4

# Good but you only get 30% of the lyrics unless you want to pay a kajillion dollars.
# Can't do track.lyrics.mood.get unless you want to pay a kajillion dollars.
# So even though I have spotify, which has musixmatch integrated I can't get my own data....

In [ ]:
# import sys
# path = "/Users/matth/Desktop/"
# sys.path.insert(0,path)
# from SpotipyConfig import MUSIXMATCH_API_KEY

In [ ]:
# raw_url = "https://api.musixmatch.com/ws/1.1/"
# # import json
# import requests

In [ ]:
# method = "track.lyrics.mood.get?track_isrc=USUG12000658"
# query = ""

In [ ]:
# response = requests.get(raw_url +method+"&apikey="+MUSIXMATCH_API_KEY)

In [ ]:
# json_data = json.loads(response.content)
# json_data

In [ ]:
# json_data["message"]["body"]["lyrics"]["lyrics_body"]

In [ ]:
# last fm https://www.last.fm/api/intro

In [ ]:
# import sys
# path = "/Users/matth/Desktop/"
# sys.path.insert(0,path)
# from SpotipyConfig import LAST_FM_API_KEY, LAST_FM_API_SHARED_SECRET

In [ ]:
# root_url = "http://ws.audioscrobbler.com/2.0/"

In [ ]:
# authenticate
# auth = "?method=auth.gettoken&api_key="+LAST_FM_API_KEY+"&format=json"

In [ ]:
# import requests
# import json

In [ ]:
# response = requests.get(root_url+auth)

In [ ]:
# json_data = json.loads(response.content)
# LAST_FM_TOKEN = json_data["token"]

In [ ]:
# import hashlib

In [ ]:
# longstring = 'api_key'+LAST_FM_API_KEY+"method"+"auth.getSession"+"token"+LAST_FM_TOKEN+LAST_FM_API_SHARED_SECRET
# LAST_FM_API_SIG = hashlib.md5(longstring.encode()).hexdigest()

In [ ]:
# auth_root_url = "http://www.last.fm/api/auth/?method=auth.getSession&api_key="+LAST_FM_API_KEY+"&token="+LAST_FM_TOKEN+"&api_sig="+LAST_FM_API_SIG

In [ ]:
# auth_root_url

In [ ]:
# response = requests.get(auth_root_url)
# # json_data = json.loads(response.content)
# response

In [ ]:
# auth_call_url = root_url

In [ ]:
# import pylast #https://github.com/pylast/pylast

In [ ]:
# import sys
# path = "/Users/matth/Desktop/"
# sys.path.insert(0,path)
# from SpotipyConfig import LAST_FM_API_KEY as API_KEY, LAST_FM_API_SHARED_SECRET as API_SECRET, LAST_FM_USERNAME as username, LAST_FM_PASSWORD as password

In [ ]:
# # In order to perform a write operation you need to authenticate yourself
# password_hash = pylast.md5(password)

# network = pylast.LastFMNetwork(
#     api_key=API_KEY,
#     api_secret=API_SECRET,
#     username=username,
#     password_hash=password_hash,
# )

In [ ]:
# # example get a track by artist and title and find out what album it's on.
# track = network.get_track("The Beatles", "if i fell")
# track.get_top_tags()

In [ ]:
# dir(track)

In [ ]:
# track.get_mbid()

In [ ]:
# MUSICBRAINZ pip install musicbrainzngs ; https://python-musicbrainzngs.readthedocs.io/en/v0.7.1/api/#general

In [ ]:
# import sys
# path = "/Users/matth/Desktop/"
# sys.path.insert(0,path)
# from SpotipyConfig import MB_USERNAME, MB_PASSWORD

In [ ]:
# import musicbrainzngs
# musicbrainzngs.set_useragent("test", "0.1")

In [ ]:
# musicbrainzngs.auth(MB_USERNAME,MB_PASSWORD)

In [ ]:
# musicbrainzngs.get_recordings_by_isrc("US95X0900010")

In [ ]:
# #test using mbid above in pylast
# network.get_track_by_mbid('b2662c42-80a7-4fdd-8fb6-97c112eff0e5')

In [ ]:
# import time
# mbids = []
# i=0
# for isrc in ext_id:
#     print(isrc)
#     try:
#         mbid_dict = musicbrainzngs.get_recordings_by_isrc(isrc["isrc"])
#         mbid = mbid_dict["isrc"]["recording-list"][0]["id"]
#     except:
#         mbid=0
#     mbids.append(mbid)
#     i=i+1
#     print(i)
#     print("-----------")
#     time.sleep(0.5)

In [ ]:
# mbids

In [ ]:
# number of not found over total is too high
# len([mbid for mbid in mbids if mbid==0])/len(mbids)

In [ ]:
# playlistFrame["mbids"]=mbids

In [ ]:
# playlistFrame[playlistFrame["mbids"]==0]

In [ ]:
# track = network.get_track("Islands", "Devout")
# track

In [ ]:
# track.title

In [ ]:
# [tag.item.name for tag in track.get_top_tags()]

In [ ]:
# lastfm_tags = []
# for i, mbid in enumerate(mbids):
#     print(artists[i]+"-"+songtitle[i])
#     try:
#         if mbid==0:
#             track = network.get_track(artists[i], songtitle[i])
#             print("search by name/title")
#         else:
#             track = network.get_track_by_mbid(mbid)
#             print("search by mbid")
#         lastfm_tags.append([tag.item.name for tag in track.get_top_tags()])
#         print("success!")
#     except:
#         lastfm_tags.append(0)
#         print("skipped..")
#     print("----------")
#     time.sleep(0.5)
# print("COMPLETE!")

In [ ]:
# len([tag for tag in lastfm_tags if tag==[]])/len(lastfm_tags)

In [ ]:
# network.get_track("OneRepublic","Sunshine").get_top_tags()

In [ ]:
## music story
# import music_story

In [ ]:
# dir(music_story)

In [ ]:
# import sys
# path = "/Users/matth/Desktop/"
# sys.path.insert(0,path)
# from SpotipyConfig import MS_KEY, MS_SECRET, MS_TOKEN, MS_TOKEN_SECRET

In [ ]:
# api = music_story.MusicStoryApi(MS_KEY, MS_SECRET, MS_TOKEN, MS_TOKEN_SECRET)
# api.connect()

In [ ]:
# genre= api.get("genre", 66)
# genre.name

In [ ]:
# releases = api.search('release', type='Live', title='love')

In [ ]:
# releases[0].id

In [ ]:
# track = api.search("track", isrc="GBUM71603956" )[0]

In [ ]:
# track.title

In [ ]:
# lyric_id = track.connector("lyric")

In [ ]:
# lyric_id=lyric_id[0].id

In [ ]:
# lyrics = api.get("lyric",lyric_id)

In [ ]:
# lyrics

In [ ]:
# track.connector("lyric").context

In [ ]:
# genre = track.connector("genre")

In [ ]:
# genre

In [ ]:
# lfind = lyrics.connector("lyricfind")

In [ ]:
# dir(lyrics)

In [ ]:
# import json

In [ ]:
# type(track.load_json)

In [ ]:
# dir(track)

In [ ]:
# azlyrics pip install azlyrics
# import azlyrics

In [ ]:
# dir(azlyrics)

In [ ]:
# print(azlyrics.songs("Oasis"))